In [1]:
# Setup
import ipynb.fs.full.indata_pro_ops
import sqlalchemy as db
from sqlalchemy import inspect
import pandas as pd
#import time


## User registration function

In [ ]:
def User_Registration(indata):
     """
        Checks if username of indata object exists and if not, creating a query that saves the username 
        as a new user in the `user` Table with a first switch/open time as `StartTime`

        Parameters
        ----------
        indata : InData object
            An object created from .json file sended from the mobile App
    """
     indata.extract()
     mode = input("To choose local database press 'l', for online press 'o'")
     while mode != 'l' and mode != 'o':
         print("Error! Try again!")
         mode = input("To choose local database press 'l', for online press 'o'")
    # In SQLAlchemy (SQL module to work with pandas) to start connection
    # first step is to create engine:
    # -----------------------------------------------------------------
     if mode == 'l':
        # For that although, the connection string is needed
         conn_str = ''
        # To connect to local database we need: user, password, port
        # and name of database:
         USR = input("Username:")
         PWD = input("Password:")
         PORT = '3306'
         DB = 'smokeout'
         SQLALCHEMY_DATABASE_URI = 'mysql://{}:{}@localhost:{}/{}'.format(USR, PWD, PORT, DB)
         engine = db.create_engine(conn_str)
     else:
        # For that although, the connection string is needed
         connAzure = "Driver={ODBC Driver 18 for SQL Server};Server=tcp:smokeout.database.windows.net,1433;Database=SmokeOut_users;Uid=Krwawe;Pwd="+input("Password")+";Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;"
        # To connect to Azure database we need a prefix
         conn_str = f'mssql+pyodbc:///?odbc_connect={connAzure}'
         engine = db.create_engine(conn_str) 

    # ===================================================================
    # Extracting the metadata (needed to access the tables)
     conn = engine.connect()
     metadata = db.MetaData()
    # Accessing `user` table of connected database 
     user = db.Table('user', metadata, autoload=True, autoload_with=engine)

    # ====================================================================
    # Checking if username is already in database and if it is not
    # inserting it in a new row of `user` table
     chkusr = False if conn.execute(db.select(user).where(user.c.username == indata.username)) is not None else True
     try:
         start = 1/chkusr
             # Reading information from "date" string
         date = indata.json['operations'][1]['date']
         try:
             hour = int(date[0:2])
             hour = '0' + str(hour) if hour < 10 else str(hour)
             minute = int(date[3:5])
             minute = '0' + str(minute) if minute < 10 else str(minute)
             second = int(date[6:8])
             second = '0' + str(second) if second < 10 else str(second)
             day = int(date[9:11])
             day = '0' + str(day) if day <10 else str(day)
             month = int(date[12:14])
             month = '0' + str(month) if month <10 else str(month)
             year = int(date[15:])
        # Constructing the string of SQL's DateTime format
             dateSQL = str(year) + "-" + month + "-" + day + " " + hour + ":" + minute + ":" + second
         except:
             print("Error occured: wrong datetime")
             raise SystemExit
         query = user.insert.values(username=indata.username, StartTime=dateSQL)
         exe = conn.execute(query)
     except:
         print("username already in database")



## Function to retrive user ID from database

In [ ]:
def get_UserID(indata, conn, user):
     """
        Check the conn'ection of `user` database for UserID of indata.username 
        and saves the UserID into UserID attribute of indata object

        Parameters
        ----------
        indata : InData object
            An object created from .json file sended from the mobile App
        conn : MockConnection
            Established connection with the db.engine
        user : Table
            db.Table structure of `user` Table from the database
    """
    # To do that, run query: SELECT column `UserID` FROM table `user` 
    # WHERE column `username` is equal to indata.username
     sfwusr = conn.execute(db.select(user.c.UserID).where(user.c.username == indata.username))
     UserID = sfwusr.fetchone() # fetchone(), fetchmany(n), fetchall() - printing a number of lines one/n-many/all
    # The format of this data is not friendly:
     print(UserID)
     for i in UserID:
         UserID = i


## Function to save extracted data into respective Tables of db

In [ ]:

def InData_save(indata, engine):
     """
        Checks if datetimes of indata object exists in db
        and if not, creating a query that saves the opening
        and switching package times into `inapp_user_data` table

        Parameters
        ----------
        indata : InData object
            An object created from .json file sended from the mobile App 
        engine : MockConnection
            SQLAlchemy engine used to establish connection with the database
    """
    # Accessing table `inapp_user_data` of database 'engine'
     conn = engine.connect()
     metadata = db.MetaData()
     inapp_user_data = db.Table('inapp_user_data', metadata, autoload=True, autoload_with=engine)
     # Let's first create lists of values for INSERT queries
     swt_list = []
     opn_list = []

     for i in indata.switch:
         swt_list.append({"UserID": indata.UserID, "SwitchPckg": i})
     print(indata.open)

     for i in indata.open:
         opn_list.append({"UserID": indata.UserID, "OpenPckg": i})
     print(opn_list)
    # Creation of an INSERT query and execution:
     query = inapp_user_data.insert()
     exe_swt = conn.execute(query, swt_list)
     exe_opn = conn.execute(query, opn_list)